Conclusion: ça permet de faire les 7 derniers jours, mais on aimerait que ça reprenne à 0 pour le mois de Septembre ! 

In [1]:
import pandas as pd
import duckdb

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
data = {
    'name': ['Toufik', 'Jean-Nicolas', 'Daniel', 'Kaouter', 'Sylvie',
             'Sebastien', 'Diane', 'Romain', 'François', 'Anna',
             'Zeinaba', 'Gregory', 'Karima', 'Arthur', 'Benjamin'],
    'wage': [60000, 75000, 55000, 80000, 70000,
             90000, 65000, 72000, 68000, 85000,
             100000, 120000, 95000, 83000, 110000],
    'department': ['IT', 'HR', 'SALES', 'IT', 'IT',
                   'HR', 'SALES', 'IT', 'HR', 'SALES',
                   'IT', 'IT', 'HR', 'SALES', 'CEO']
}
wages = pd.DataFrame(data)
wages.assign(
    max_per_dpt = lambda df_: df_[["department", "wage"]].groupby("department").transform("max")
).sort_values("department")

,name,wage,department,max_per_dpt
14,Benjamin,110000,CEO,110000
1,Jean-Nicolas,75000,HR,95000
5,Sebastien,90000,HR,95000
8,François,68000,HR,95000
12,Karima,95000,HR,95000
0,Toufik,60000,IT,120000
3,Kaouter,80000,IT,120000
4,Sylvie,70000,IT,120000
7,Romain,72000,IT,120000
10,Zeinaba,100000,IT,120000


On aimerait savoir quel est le plus gros salaire par département ?

In [3]:
query = """
SELECT department,
MAX(wage)
FROM wages
GROUP BY department
"""
duckdb.sql(query).df()

,department,max(wage)
0,HR,95000
1,CEO,110000
2,SALES,85000
3,IT,120000


Maintenant, on aimerait l'avoir en face de chaque salaire pour calculer <br />
les écarts de salaires entre les employés d'un même département

## La syntaxe "PARTITION BY"

Il faut imaginer que vous faites une "sous-table" pour chaque département, <br />
et un MAX(wage) OVER() pour chacune de ces sous-tables

In [4]:
for department in wages["department"].unique().tolist():
    subset = wages[ wages["department"] == department ]
    query = """
    SELECT *,
    MAX(wage) OVER() AS max_dpt_wage
    FROM subset
    ORDER BY department
    """
    display(duckdb.sql(query).df())

,name,wage,department,max_dpt_wage
0,Toufik,60000,IT,120000
1,Kaouter,80000,IT,120000
2,Sylvie,70000,IT,120000
3,Romain,72000,IT,120000
4,Zeinaba,100000,IT,120000
5,Gregory,120000,IT,120000


,name,wage,department,max_dpt_wage
0,Jean-Nicolas,75000,HR,95000
1,Sebastien,90000,HR,95000
2,François,68000,HR,95000
3,Karima,95000,HR,95000


,name,wage,department,max_dpt_wage
0,Daniel,55000,SALES,85000
1,Diane,65000,SALES,85000
2,Anna,85000,SALES,85000
3,Arthur,83000,SALES,85000


,name,wage,department,max_dpt_wage
0,Benjamin,110000,CEO,110000


Pour faire ça efficacement, on peut utiliser la clause OVER(PARTITION BY ...)

In [6]:
query = """
SELECT *,
MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage
FROM wages
ORDER BY department
"""
duckdb.sql(query).df()

,name,wage,department,max_dpt_wage
0,Benjamin,110000,CEO,110000
1,Jean-Nicolas,75000,HR,95000
2,Sebastien,90000,HR,95000
3,François,68000,HR,95000
4,Karima,95000,HR,95000
5,Toufik,60000,IT,120000
6,Kaouter,80000,IT,120000
7,Sylvie,70000,IT,120000
8,Romain,72000,IT,120000
9,Zeinaba,100000,IT,120000


In [7]:
wages.assign(
    max_wage= lambda df_: 
                df_.groupby(["department"])
                .transform("max")["wage"]
).sort_values("department")

,name,wage,department,max_wage
14,Benjamin,110000,CEO,110000
1,Jean-Nicolas,75000,HR,95000
5,Sebastien,90000,HR,95000
8,François,68000,HR,95000
12,Karima,95000,HR,95000
0,Toufik,60000,IT,120000
3,Kaouter,80000,IT,120000
4,Sylvie,70000,IT,120000
7,Romain,72000,IT,120000
10,Zeinaba,100000,IT,120000


In [8]:
wages.groupby(["department"]).transform("max")

,name,wage
0,Zeinaba,120000
1,Sebastien,95000
2,Diane,85000
3,Zeinaba,120000
4,Zeinaba,120000
5,Sebastien,95000
6,Diane,85000
7,Zeinaba,120000
8,Sebastien,95000
9,Diane,85000


## Exercice:
Calculez le salaire moyen par department

In [11]:
query = """
SELECT
    *,
    AVG(wage) OVER(PARTITION BY department) AS mean_dpt_wage
FROM wages
ORDER BY mean_dpt_wage DESC

"""
duckdb.sql(query).df()

,name,wage,department,mean_dpt_wage
0,Benjamin,110000,CEO,110000.000000
1,Toufik,60000,IT,83666.666667
2,Kaouter,80000,IT,83666.666667
3,Sylvie,70000,IT,83666.666667
4,Romain,72000,IT,83666.666667
5,Zeinaba,100000,IT,83666.666667
6,Gregory,120000,IT,83666.666667
7,Jean-Nicolas,75000,HR,82000.000000
8,Sebastien,90000,HR,82000.000000
9,François,68000,HR,82000.000000


In [ ]:
# %load solutions/8partition_by_simple.py
query = """
SELECT *,
AVG(wage) OVER(PARTITION BY department) AS mean_dpt_wage
FROM wages
ORDER BY department
"""
duckdb.sql(query).df()


Créez une nouvelle colonne qui indique si un salarié est au max de son département

In [16]:
query = """
SELECT *,
MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage,
wage >= max_dpt_wage AS is_max
FROM wages
ORDER BY department
"""
duckdb.sql(query).df()

,name,wage,department,max_dpt_wage,is_max
0,Benjamin,110000,CEO,110000,True
1,Jean-Nicolas,75000,HR,95000,False
2,Sebastien,90000,HR,95000,False
3,François,68000,HR,95000,False
4,Karima,95000,HR,95000,True
5,Toufik,60000,IT,120000,False
6,Kaouter,80000,IT,120000,False
7,Sylvie,70000,IT,120000,False
8,Romain,72000,IT,120000,False
9,Zeinaba,100000,IT,120000,False


In [17]:
# %load solutions/9max_per_dpt.py
query = """
SELECT *,
MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage,
wage >= max_dpt_wage AS is_max
FROM wages
ORDER BY department
"""
duckdb.sql(query).df()


,name,wage,department,max_dpt_wage,is_max
0,Benjamin,110000,CEO,110000,True
1,Jean-Nicolas,75000,HR,95000,False
2,Sebastien,90000,HR,95000,False
3,François,68000,HR,95000,False
4,Karima,95000,HR,95000,True
5,Toufik,60000,IT,120000,False
6,Kaouter,80000,IT,120000,False
7,Sylvie,70000,IT,120000,False
8,Romain,72000,IT,120000,False
9,Zeinaba,100000,IT,120000,False


Servez-vous de ça pour calculer le 2e MAX. 

Indice: utilisez une CTE pour stocker le 1er salaire maximum par département, <br />
puis filtrer cette CTE pour exclure le MAX et refaites la même procédure <br />
pour trouver le nouveau MAX sur les records restants

In [23]:
query = """
WITH max_salaries_per_dpt AS(
SELECT
    *,
    MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage,
    wage >= max_dpt_wage AS max_salary
FROM wages
ORDER BY department
-- on va garder que les max_salary = False
)

SELECT
    *,
    MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage
FROM max_salaries_per_dpt
WHERE max_salary = False

"""
duckdb.sql(query).df()

,name,wage,department,max_dpt_wage,max_salary,max_dpt_wage_1
0,Jean-Nicolas,75000,HR,95000,False,90000
1,Sebastien,90000,HR,95000,False,90000
2,François,68000,HR,95000,False,90000
3,Arthur,83000,SALES,85000,False,83000
4,Daniel,55000,SALES,85000,False,83000
5,Diane,65000,SALES,85000,False,83000
6,Sylvie,70000,IT,120000,False,100000
7,Romain,72000,IT,120000,False,100000
8,Zeinaba,100000,IT,120000,False,100000
9,Toufik,60000,IT,120000,False,100000


In [24]:
# %load solutions/10second_max_per_dpt.py
query = """
WITH max_salaries_per_dpt AS (
    SELECT *,
    MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage,
    wage >= max_dpt_wage AS max_salary,
    FROM wages
    -- QUALIFY max_salary = False
    ORDER BY department
)


SELECT *,
MAX(wage) OVER(PARTITION BY department) AS max_dpt_wage
FROM max_salaries_per_dpt
WHERE max_salary = False
"""
duckdb.sql(query).df()


,name,wage,department,max_dpt_wage,max_salary,max_dpt_wage_1
0,Jean-Nicolas,75000,HR,95000,False,90000
1,Sebastien,90000,HR,95000,False,90000
2,François,68000,HR,95000,False,90000
3,Toufik,60000,IT,120000,False,100000
4,Kaouter,80000,IT,120000,False,100000
5,Sylvie,70000,IT,120000,False,100000
6,Romain,72000,IT,120000,False,100000
7,Zeinaba,100000,IT,120000,False,100000
8,Daniel,55000,SALES,85000,False,83000
9,Diane,65000,SALES,85000,False,83000


## Résolution du problème initial

In [25]:
capteurs = pd.read_csv("data/capteur_a_retrail.csv")
capteurs.head(6)

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct
0,2023-08-01,porte_a,4200.0,3,4920.0,3936.0
1,2023-08-02,porte_a,5300.0,4,4920.0,3936.0
2,2023-08-03,porte_a,4400.0,5,4920.0,3936.0
3,2023-08-04,porte_a,5500.0,6,4920.0,3936.0
4,2023-08-05,porte_a,6000.0,7,4920.0,3936.0
5,2023-08-07,porte_a,4200.0,2,4920.0,3936.0


Mettez en face de chaque visiteurs_count la moyenne sur les 7 derniers jours, <br />
en partitionnant la table en fonction du jour de la semaine, <br />
puis multipliez cette colonne par 0.8 pour créer votre nouveau seuil

In [29]:
query = """
SELECT
    *,
    AVG(visiteurs_count) OVER(PARTITION BY weekday ROWS BETWEEN 6 PRECEDING and CURRENT ROW) AS avg_last_seven_same_days,
    avg_last_seven_same_days * 0.8 AS threshold
FROM capteurs

"""
duckdb.sql(query).df()

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000,4800.000000
1,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000,4800.000000
2,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333,4666.666667
3,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000,4700.000000
4,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000,4400.000000
5,2023-08-07,porte_a,4200.0,2,4920.0,3936.0,4200.000000,3360.000000
6,2023-08-14,porte_a,3900.0,2,4920.0,3936.0,4050.000000,3240.000000
7,2023-08-21,porte_a,4300.0,2,4920.0,3936.0,4133.333333,3306.666667
8,2023-08-28,porte_a,4300.0,2,4920.0,3936.0,4175.000000,3340.000000
9,2023-09-04,porte_a,4500.0,2,4700.0,3760.0,4240.000000,3392.000000


In [30]:
# %load solutions/11partition_by_weekday.py
query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY weekday 
    ROWS BETWEEN 6 PRECEDING 
    AND CURRENT ROW
    ) AS avg_last_seven_same_days,
avg_last_seven_same_days * 0.8 as threshold
FROM capteurs
"""
df = duckdb.sql(query).df()
df


,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-03,porte_a,4400.0,5,4920.0,3936.0,4400.000000,3520.000000
1,2023-08-10,porte_a,4400.0,5,4920.0,3936.0,4400.000000,3520.000000
2,2023-08-17,porte_a,3600.0,5,4920.0,3936.0,4133.333333,3306.666667
3,2023-08-24,porte_a,4700.0,5,4920.0,3936.0,4275.000000,3420.000000
4,2023-08-31,porte_a,4700.0,5,4920.0,3936.0,4360.000000,3488.000000
5,2023-08-01,porte_a,4200.0,3,4920.0,3936.0,4200.000000,3360.000000
6,2023-08-08,porte_a,4700.0,3,4920.0,3936.0,4450.000000,3560.000000
7,2023-08-15,porte_a,4200.0,3,4920.0,3936.0,4366.666667,3493.333333
8,2023-08-22,porte_a,4900.0,3,4920.0,3936.0,4500.000000,3600.000000
9,2023-08-29,porte_a,4750.0,3,4920.0,3936.0,4550.000000,3640.000000


Vérification: le premier Samedi de Septembre devrait déclencher une alerte:

In [31]:
colors = {
    1:'background-color: red',
}


def highlight_day(s):
    return [colors.get(s.loc['visiteurs_count'] < s.loc['threshold']) for v in s.index]


df.style.apply(highlight_day, axis=1)

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-03,porte_a,4400.000000,5,4920.000000,3936.000000,4400.000000,3520.000000
1,2023-08-10,porte_a,4400.000000,5,4920.000000,3936.000000,4400.000000,3520.000000
2,2023-08-17,porte_a,3600.000000,5,4920.000000,3936.000000,4133.333333,3306.666667
3,2023-08-24,porte_a,4700.000000,5,4920.000000,3936.000000,4275.000000,3420.000000
4,2023-08-31,porte_a,4700.000000,5,4920.000000,3936.000000,4360.000000,3488.000000
5,2023-08-01,porte_a,4200.000000,3,4920.000000,3936.000000,4200.000000,3360.000000
6,2023-08-08,porte_a,4700.000000,3,4920.000000,3936.000000,4450.000000,3560.000000
7,2023-08-15,porte_a,4200.000000,3,4920.000000,3936.000000,4366.666667,3493.333333
8,2023-08-22,porte_a,4900.000000,3,4920.000000,3936.000000,4500.000000,3600.000000
9,2023-08-29,porte_a,4750.000000,3,4920.000000,3936.000000,4550.000000,3640.000000


## Multiples clauses dans le OVER()

Attention ! 

Il se trouve que la dataframe était ordonée par date, mais rien ne garantit que ce sera toujours le cas! 

Que se passerait-il si les jours n'étaient pas correctement ordonnés?

In [32]:
shuffled = capteurs.sample(len(capteurs))
query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY weekday 
    ROWS BETWEEN 6 PRECEDING 
    AND CURRENT ROW
    ) AS avg_last_seven_same_days,
avg_last_seven_same_days * 0.8 as threshold
FROM shuffled
WHERE weekday = 7
-- ORDER BY date
"""
df = duckdb.sql(query).df()
df

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5500.000000,4400.000000
1,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,5750.000000,4600.000000
2,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,5833.333333,4666.666667
3,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5375.000000,4300.000000
4,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5500.000000,4400.000000


Pour être sur que les données sont correctement triées <b> avant </b> de calculer les indicateurs, il faut utiliser ORDER BY _à l'intérieur_ de la clause OVER.

Vous savez comment faire ça, à vous de jouer:

In [34]:
query = """
SELECT
    *,
    AVG(visiteurs_count) OVER(PARTITION BY weekday ORDER BY date ROWS BETWEEN 6 PRECEDING and CURRENT ROW) AS avg_last_seven_same_days,
    avg_last_seven_same_days * 0.8 AS threshold
FROM shuffled

"""
duckdb.sql(query).df()

,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-03,porte_a,4400.0,5,4920.0,3936.0,4400.000000,3520.000000
1,2023-08-10,porte_a,4400.0,5,4920.0,3936.0,4400.000000,3520.000000
2,2023-08-17,porte_a,3600.0,5,4920.0,3936.0,4133.333333,3306.666667
3,2023-08-24,porte_a,4700.0,5,4920.0,3936.0,4275.000000,3420.000000
4,2023-08-31,porte_a,4700.0,5,4920.0,3936.0,4360.000000,3488.000000
5,2023-08-07,porte_a,4200.0,2,4920.0,3936.0,4200.000000,3360.000000
6,2023-08-14,porte_a,3900.0,2,4920.0,3936.0,4050.000000,3240.000000
7,2023-08-21,porte_a,4300.0,2,4920.0,3936.0,4133.333333,3306.666667
8,2023-08-28,porte_a,4300.0,2,4920.0,3936.0,4175.000000,3340.000000
9,2023-09-04,porte_a,4500.0,2,4700.0,3760.0,4240.000000,3392.000000


In [35]:
# %load solutions/12partitionby_orderby.py
shuffled = capteurs.sample(len(capteurs))
query = """
SELECT *,
AVG(visiteurs_count) OVER(
    PARTITION BY weekday 
    ORDER BY date
    ROWS BETWEEN 6 PRECEDING 
    AND CURRENT ROW
    ) AS avg_last_seven_same_days,
avg_last_seven_same_days * 0.8 as threshold
FROM shuffled
WHERE weekday = 7
"""
df = duckdb.sql(query).df()
df


,date,capteur_id,visiteurs_count,weekday,moyenne_du_mois,threshold_twenty_pct,avg_last_seven_same_days,threshold
0,2023-08-05,porte_a,6000.0,7,4920.0,3936.0,6000.000000,4800.000000
1,2023-08-12,porte_a,6000.0,7,4920.0,3936.0,6000.000000,4800.000000
2,2023-08-19,porte_a,5500.0,7,4920.0,3936.0,5833.333333,4666.666667
3,2023-08-26,porte_a,6000.0,7,4920.0,3936.0,5875.000000,4700.000000
4,2023-09-02,porte_a,4000.0,7,4700.0,3760.0,5500.000000,4400.000000


### Conclusion

On a résolu le problème initial: récupérer la moyenne glissante (ou mobile) <br />
sur les 7 derniers jours similaires <br />
pour comparer les Mardi avec les Mardi et les Samedi avec les Samedi <br />
et avoir des seuils qui ont du sens

<blockquote> Ouais, par contre ta requête pour avoir le 2e meilleur salaire par département, il n'y aurait pas moyen de l'améliorer un peu ? </blockquote>

Si ! On va voir avec quelle clause dans le prochain notebook ;) 